# Q5: Trial Duration Analysis

## Research Question

> **What is the typical duration of trials by phase and therapeutic area? Which trials take significantly longer than expected?**

## Key Methodological Note

**Why survival analysis?** Trial duration data is **right-censored**: active trials have started but not yet completed. Naive averages of completed trials only would underestimate true duration (survivorship bias). Survival analysis (Kaplan-Meier, Cox regression) properly handles censoring.

**What we exclude:** Stopped trials (Terminated, Withdrawn, Suspended) are excluded from time-to-completion analysis because `completion_date` ≠ actual stop date—we don't know when they actually stopped.

## Analysis Structure

| Section | Question | Method |
|---------|----------|--------|
| **2. Descriptive** | What is typical duration? | Kaplan-Meier median + **RMST** (area under curve) |
| **3. By Phase** | How does duration vary by phase? | KM curves + log-rank + **RMST comparison** |
| **4. By Therapeutic Area** | Which conditions take longest? | Stratified KM + **RMST ranking** |
| **5. Predictive Model** | What factors predict duration? | Cox Proportional Hazards regression |
| **6. Outliers** | Which trials are slower than expected? | Deviance residuals from Cox model |

**Key metric: RMST (Restricted Mean Survival Time)** = area under the Kaplan-Meier curve up to time τ. Unlike median, RMST uses all data points and provides an interpretable "expected duration" in days/years. Differences are directly comparable (e.g., "Phase 3 takes 1.2 years longer than Phase 1").

## Scope & Data Notes

- **Data source:** ClinicalTrials.gov (extracted via API)
- **Temporal filter:** start_year 1990–2025 (defined in `v_studies_clean.is_start_year_in_scope`)
- **Duration definition:** Start date → completion date (or extraction date for active trials)
- **Censoring:** Active trials are right-censored at extraction date
- **Exclusions:** Stopped trials (no reliable stop date), studies missing start date
- **Interpretation:** Associations not causal; duration reflects registry data, not operational reality

In [1]:
# ============================================================
# Setup
# ============================================================

import sys
from pathlib import Path
from datetime import date

import numpy as np
import pandas as pd
from scipy.stats import mannwhitneyu, kruskal, spearmanr
from IPython.display import display, Markdown
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Survival analysis
from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.statistics import logrank_test, multivariate_logrank_test

# Project root for imports
PROJECT_ROOT = Path('..')
sys.path.insert(0, str(PROJECT_ROOT))

# Shared utilities
from src.data.loader import load_sql_query, get_db_connection
from src.analysis.viz import DEFAULT_COLORS, create_horizontal_bar_chart
from src.analysis.metrics import interpret_effect_size
from src.analysis.constants import PHASE_ORDER_CLINICAL, COHORT_BINS, COHORT_LABELS

# Paths (validated at setup)
DB_PATH = PROJECT_ROOT / 'data' / 'database' / 'clinical_trials.db'
SQL_PATH = PROJECT_ROOT / 'sql' / 'queries'
assert DB_PATH.exists(), f"DB not found: {DB_PATH}"
assert SQL_PATH.exists(), f"SQL folder not found: {SQL_PATH}"

# Analysis parameters
EXTRACTION_DATE = '2025-01-20'  # Date of data extraction for censoring
RMST_HORIZON_YEARS = 5  # Restricted mean survival time horizon (τ)
RMST_HORIZON_DAYS = RMST_HORIZON_YEARS * 365.25

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [2]:
# ============================================================
# Database connection
# ============================================================

conn = get_db_connection(DB_PATH)

---

## 1. Data Loading & Validation

In [3]:
# ============================================================
# 1.1 Load ABT
# ============================================================

df_abt = load_sql_query(
    'q5_abt.sql', 
    conn, 
    SQL_PATH,
    params={'extraction_date': EXTRACTION_DATE}
)

# Basic validation
n_total = len(df_abt)
n_completed = df_abt['is_completed'].sum()
n_active = df_abt['is_active'].sum()
n_stopped = df_abt['is_stopped'].sum()

# Valid for survival analysis
n_valid_survival = df_abt['is_valid_for_survival'].sum()
pct_valid = n_valid_survival / n_total * 100

# Duration coverage
n_with_duration = df_abt['duration_days'].notna().sum()

display(Markdown(f"""
### 1.1 Data loaded

**ABT:** {n_total:,} studies (extraction date: {EXTRACTION_DATE})

| Status | N | % | Notes |
|--------|---|---|-------|
| Completed | {n_completed:,} | {n_completed/n_total*100:.1f}% | Event observed |
| Active | {n_active:,} | {n_active/n_total*100:.1f}% | Right-censored |
| Stopped | {n_stopped:,} | {n_stopped/n_total*100:.1f}% | **Excluded** (no stop date) |

**Valid for survival analysis:** {n_valid_survival:,} ({pct_valid:.1f}%)

*Stopped trials are excluded because `completion_date` ≠ actual stop date.*
"""))


### 1.1 Data loaded

**ABT:** 82,707 studies (extraction date: 2025-01-20)

| Status | N | % | Notes |
|--------|---|---|-------|
| Completed | 54,184 | 65.5% | Event observed |
| Active | 19,749 | 23.9% | Right-censored |
| Stopped | 8,774 | 10.6% | **Excluded** (no stop date) |

**Valid for survival analysis:** 73,698 (89.1%)

*Stopped trials are excluded because `completion_date` ≠ actual stop date.*


In [4]:
# ============================================================
# 1.2 Filter to valid survival analysis population
# ============================================================

# Survival analysis population: completed + active with valid duration
df_surv = df_abt[
    (df_abt['is_valid_for_survival'] == 1) & 
    (df_abt['duration_days'].notna()) &
    (df_abt['duration_days'] > 0)  # Exclude zero/negative durations
].copy()

n_surv = len(df_surv)
n_events = df_surv['event_completed'].sum()
n_censored = n_surv - n_events
pct_censored = n_censored / n_surv * 100

# Duration statistics (descriptive - completed only)
df_completed = df_surv[df_surv['event_completed'] == 1]
median_days_completed = df_completed['duration_days'].median()
median_years_completed = median_days_completed / 365.25

display(Markdown(f"""
### 1.2 Survival analysis population

**Population:** {n_surv:,} studies
- Events (completed): {n_events:,}
- Censored (active): {n_censored:,} ({pct_censored:.1f}%)

**Naive duration (completed only):**
- Median: {median_days_completed:.0f} days ({median_years_completed:.1f} years)

*Naive median excludes censored trials. KM estimate (Section 2) accounts for right-censoring.*
"""))


### 1.2 Survival analysis population

**Population:** 67,938 studies
- Events (completed): 53,706.0
- Censored (active): 14,232.0 (20.9%)

**Naive duration (completed only):**
- Median: 658 days (1.8 years)

*Naive median excludes censored trials. KM estimate (Section 2) accounts for right-censoring.*


---

## 2. Overall Duration Distribution

**Question:** What is the typical duration of clinical trials?

In [5]:
# ============================================================
# 2.1 Kaplan-Meier estimate (overall)
# ============================================================

# Create timeline for smooth survival function (daily resolution up to 15 years)
timeline = np.arange(0, 15 * 365.25 + 1, 1)  # Daily steps

# Fit Kaplan-Meier with explicit timeline
kmf = KaplanMeierFitter()
kmf.fit(
    durations=df_surv['duration_days'],
    event_observed=df_surv['event_completed'],
    timeline=timeline,
    label='All Trials'
)

# Extract key statistics
km_median_days = kmf.median_survival_time_
km_median_years = km_median_days / 365.25 if not np.isinf(km_median_days) else np.inf

# Percentiles from survival function
km_25pct = kmf.percentile(0.75)  # 75% survival = 25% have completed
km_75pct = kmf.percentile(0.25)  # 25% survival = 75% have completed

# ============================================================
# RMST: Restricted Mean Survival Time (area under the curve)
# ============================================================
# RMST = expected duration up to time τ, accounting for censoring
# More interpretable than median when comparing groups

def calc_rmst(kmf_fitted, tau):
    """Calculate RMST (area under KM curve) up to time tau."""
    sf = kmf_fitted.survival_function_
    sf_restricted = sf[sf.index <= tau].copy()
    if tau not in sf_restricted.index:
        sf_restricted.loc[tau] = kmf_fitted.predict(tau)
        sf_restricted = sf_restricted.sort_index()
    times = sf_restricted.index.values
    probs = sf_restricted.values.flatten()
    return np.trapezoid(probs, times)

def calc_rmst_from_data(durations, events, tau, timeline):
    """Calculate RMST from raw data (for bootstrap)."""
    kmf_temp = KaplanMeierFitter()
    kmf_temp.fit(durations, event_observed=events, timeline=timeline)
    return calc_rmst(kmf_temp, tau)

def bootstrap_rmst_ci(durations, events, tau, timeline, n_bootstrap=1000, alpha=0.05):
    """Bootstrap CI for RMST."""
    np.random.seed(42)
    n = len(durations)
    rmst_boot = []
    for _ in range(n_bootstrap):
        idx = np.random.choice(n, size=n, replace=True)
        rmst_boot.append(calc_rmst_from_data(durations.iloc[idx], events.iloc[idx], tau, timeline))
    ci_low = np.percentile(rmst_boot, 100 * alpha / 2)
    ci_high = np.percentile(rmst_boot, 100 * (1 - alpha / 2))
    return np.mean(rmst_boot), ci_low, ci_high

# Calculate RMST with bootstrap CI
rmst_days = calc_rmst(kmf, RMST_HORIZON_DAYS)
rmst_years = rmst_days / 365.25

# Bootstrap CI (subsample for computational efficiency)
# Justification: With n > 10k, bootstrap variance stabilizes; additional samples add 
# runtime without meaningfully narrowing CIs. Subsample preserves event/censoring ratio.
n_boot_sample = min(len(df_surv), 10000)
if len(df_surv) > n_boot_sample:
    df_boot = df_surv.sample(n=n_boot_sample, random_state=42)
else:
    df_boot = df_surv
_, rmst_ci_low, rmst_ci_high = bootstrap_rmst_ci(
    df_boot['duration_days'], df_boot['event_completed'], 
    RMST_HORIZON_DAYS, timeline, n_bootstrap=500
)
rmst_ci_low_years = rmst_ci_low / 365.25
rmst_ci_high_years = rmst_ci_high / 365.25

display(Markdown(f"""
### 2.1 Duration estimates

| Metric | Estimate | 95% CI | Interpretation |
|--------|----------|--------|----------------|
| **KM Median** | {km_median_years:.2f} years | — | 50% complete by this time |
| **RMST (τ={RMST_HORIZON_YEARS}y)** | **{rmst_years:.2f} years** | [{rmst_ci_low_years:.2f}, {rmst_ci_high_years:.2f}] | Expected duration within {RMST_HORIZON_YEARS}y horizon |

**RMST interpretation:** Within a {RMST_HORIZON_YEARS}-year window, trials take on average {rmst_years:.1f} years to complete (95% CI: {rmst_ci_low_years:.2f}–{rmst_ci_high_years:.2f}). The narrow CI reflects the large sample size (n={n_surv:,}).

**Naive vs KM median:** Completed-only median ({median_days_completed/365.25:.1f}y) < KM median ({km_median_years:.1f}y) because KM accounts for censored trials still running.
"""))

# RMST sensitivity to different τ horizons
display(Markdown("### 2.1.1 RMST sensitivity to horizon τ"))
tau_values = [3, 5, 7, 10]
tau_sensitivity = []
for tau_y in tau_values:
    tau_d = tau_y * 365.25
    rmst_tau = calc_rmst(kmf, tau_d) / 365.25
    # What % of KM curve is captured?
    pct_captured = (1 - kmf.predict(tau_d)) * 100 if tau_d <= kmf.survival_function_.index.max() else 100
    tau_sensitivity.append({
        'τ (years)': tau_y,
        'RMST (years)': f"{rmst_tau:.2f}",
        '% completed by τ': f"{pct_captured:.0f}%",
    })

display(pd.DataFrame(tau_sensitivity).style.hide(axis='index'))
display(Markdown(f"""
*At τ={RMST_HORIZON_YEARS}y, ~{(1-kmf.predict(RMST_HORIZON_DAYS))*100:.0f}% of trials have completed—a balance between coverage and interpretability.*
"""))

/Users/pedro/work/Clinical-Trial-Analytics/.venv/lib/python3.12/site-packages/lifelines/fitters/__init__.py:279: ApproximationWarning: Approximating using `survival_function_`. To increase accuracy, try using or increasing the resolution of the timeline kwarg in `.fit(..., timeline=timeline)`.

  warnings.warn(
/Users/pedro/work/Clinical-Trial-Analytics/.venv/lib/python3.12/site-packages/lifelines/fitters/__init__.py:279: ApproximationWarning: Approximating using `survival_function_`. To increase accuracy, try using or increasing the resolution of the timeline kwarg in `.fit(..., timeline=timeline)`.

  warnings.warn(



### 2.1 Duration estimates

| Metric | Estimate | 95% CI | Interpretation |
|--------|----------|--------|----------------|
| **KM Median** | 2.21 years | — | 50% complete by this time |
| **RMST (τ=5y)** | **2.54 years** | [2.52, 2.59] | Expected duration within 5y horizon |

**RMST interpretation:** Within a 5-year window, trials take on average 2.5 years to complete (95% CI: 2.52–2.59). The narrow CI reflects the large sample size (n=67,938).

**Naive vs KM median:** Completed-only median (1.8y) < KM median (2.2y) because KM accounts for censored trials still running.


### 2.1.1 RMST sensitivity to horizon τ

τ (years),RMST (years),% completed by τ
3,1.97,61%
5,2.54,80%
7,2.84,89%
10,3.08,94%



*At τ=5y, ~80% of trials have completed—a balance between coverage and interpretability.*


In [6]:
# ============================================================
# 2.2 Kaplan-Meier survival curve (overall) with 95% CI
# ============================================================

# Get survival function and confidence intervals
sf = kmf.survival_function_
ci = kmf.confidence_interval_survival_function_

# Convert to years for interpretability
sf_years = sf.copy()
sf_years.index = sf_years.index / 365.25

ci_years = ci.copy()
ci_years.index = ci_years.index / 365.25

# Create plot
fig_km = go.Figure()

# Add confidence interval band (first, so it's behind the line)
fig_km.add_trace(go.Scatter(
    x=list(ci_years.index) + list(ci_years.index[::-1]),
    y=list(ci_years.iloc[:, 1] * 100) + list(ci_years.iloc[:, 0][::-1] * 100),
    fill='toself',
    fillcolor='rgba(99, 102, 241, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo='skip',
    showlegend=True,
    name='95% CI',
))

# Add main survival line
fig_km.add_trace(go.Scatter(
    x=sf_years.index,
    y=sf_years['All Trials'] * 100,
    mode='lines',
    name='Survival probability',
    line=dict(color=DEFAULT_COLORS[0], width=2),
    hovertemplate='Year: %{x:.1f}<br>Still running: %{y:.1f}%<extra></extra>',
))

# Add median line
if not np.isinf(km_median_years):
    fig_km.add_hline(y=50, line_dash='dash', line_color='gray', opacity=0.5)
    fig_km.add_vline(x=km_median_years, line_dash='dash', line_color='gray', opacity=0.5)
    fig_km.add_annotation(
        x=km_median_years, y=55,
        text=f'Median: {km_median_years:.1f} years',
        showarrow=False, font=dict(size=11)
    )

fig_km.update_layout(
    title=dict(
        text='<b>Kaplan-Meier Survival Curve: Time to Trial Completion</b><br>'
             f'<span style="font-size:12px;color:gray">n = {n_surv:,} | {n_censored:,} censored ({pct_censored:.0f}%) | Shaded = 95% CI</span>',
        x=0.5, xanchor='center'
    ),
    xaxis=dict(title='Years since start', range=[0, min(15, sf_years.index.max())]),
    yaxis=dict(title='% of trials still running', range=[0, 105]),
    template='plotly_white',
    height=450,
    legend=dict(x=0.95, y=0.95, xanchor='right'),
)

fig_km.show()

display(Markdown("""
*The shaded band shows the 95% confidence interval (Greenwood's formula). The CI is narrow because of the large sample size.*
"""))


*The shaded band shows the 95% confidence interval (Greenwood's formula). The CI is narrow because of the large sample size.*


---

## 3. Duration by Phase

**Question:** How does trial duration vary by development phase?

In [7]:
# ============================================================
# 3.1 Kaplan-Meier by phase
# ============================================================

# Filter to interventional trials with clinical phases
clinical_phases = ['Phase 1', 'Phase 2', 'Phase 3', 'Phase 4']
df_phase = df_surv[
    (df_surv['is_interventional'] == 1) &
    (df_surv['phase_group'].isin(clinical_phases))
].copy()

# Fit KM for each phase with RMST and bootstrap CI
phase_km = {}
phase_data_dict = {}
phase_stats = []

for phase in clinical_phases:
    phase_data = df_phase[df_phase['phase_group'] == phase]
    if len(phase_data) >= 50:
        kmf_phase = KaplanMeierFitter()
        kmf_phase.fit(
            durations=phase_data['duration_days'],
            event_observed=phase_data['event_completed'],
            timeline=timeline,
            label=phase
        )
        phase_km[phase] = kmf_phase
        phase_data_dict[phase] = phase_data
        
        median_years = kmf_phase.median_survival_time_ / 365.25
        rmst_phase_years = calc_rmst(kmf_phase, RMST_HORIZON_DAYS) / 365.25
        
        # Bootstrap CI for RMST (subsample for speed)
        n_sub = min(len(phase_data), 3000)
        pd_sub = phase_data.sample(n=n_sub, random_state=42) if len(phase_data) > n_sub else phase_data
        _, ci_low, ci_high = bootstrap_rmst_ci(
            pd_sub['duration_days'], pd_sub['event_completed'],
            RMST_HORIZON_DAYS, timeline, n_bootstrap=300
        )
        
        phase_stats.append({
            'Phase': phase,
            'N': len(phase_data),
            'Events': int(phase_data['event_completed'].sum()),
            'RMST (years)': rmst_phase_years,
            'RMST 95% CI': f"[{ci_low/365.25:.2f}, {ci_high/365.25:.2f}]",
        })

df_phase_stats = pd.DataFrame(phase_stats)

# RMST difference test: Phase 3 vs Phase 1 (bootstrap)
def bootstrap_rmst_diff(data1, data2, tau, timeline, n_bootstrap=500):
    """Bootstrap test for RMST difference between two groups."""
    np.random.seed(43)
    diffs = []
    for _ in range(n_bootstrap):
        idx1 = np.random.choice(len(data1), len(data1), replace=True)
        idx2 = np.random.choice(len(data2), len(data2), replace=True)
        rmst1 = calc_rmst_from_data(data1['duration_days'].iloc[idx1], data1['event_completed'].iloc[idx1], tau, timeline)
        rmst2 = calc_rmst_from_data(data2['duration_days'].iloc[idx2], data2['event_completed'].iloc[idx2], tau, timeline)
        diffs.append(rmst2 - rmst1)
    diff_est = np.mean(diffs)
    diff_ci_low = np.percentile(diffs, 2.5)
    diff_ci_high = np.percentile(diffs, 97.5)
    # p-value: proportion of bootstrap samples with opposite sign
    p_val = 2 * min(np.mean(np.array(diffs) > 0), np.mean(np.array(diffs) < 0))
    return diff_est, diff_ci_low, diff_ci_high, p_val

if 'Phase 1' in phase_data_dict and 'Phase 3' in phase_data_dict:
    p1_data = phase_data_dict['Phase 1'].sample(n=min(2000, len(phase_data_dict['Phase 1'])), random_state=44)
    p3_data = phase_data_dict['Phase 3'].sample(n=min(2000, len(phase_data_dict['Phase 3'])), random_state=44)
    diff_days, diff_ci_low, diff_ci_high, diff_p = bootstrap_rmst_diff(p1_data, p3_data, RMST_HORIZON_DAYS, timeline, 300)
    rmst_diff_years = diff_days / 365.25
    rmst_diff_ci = f"[{diff_ci_low/365.25:.2f}, {diff_ci_high/365.25:.2f}]"
    diff_sig = "significant" if diff_p < 0.05 else "not significant"
else:
    rmst_diff_years, rmst_diff_ci, diff_p, diff_sig = 0, "N/A", 1, "N/A"

# Add delta column
rmst_p1 = df_phase_stats[df_phase_stats['Phase'] == 'Phase 1']['RMST (years)'].values[0]
df_phase_stats['Δ vs P1'] = df_phase_stats['RMST (years)'].apply(lambda x: f"{x - rmst_p1:+.2f}y")

display(Markdown("### 3.1 Duration by phase"))
display(df_phase_stats[['Phase', 'N', 'Events', 'RMST (years)', 'RMST 95% CI', 'Δ vs P1']].style.format({
    'RMST (years)': '{:.2f}'
}).hide(axis='index'))

display(Markdown(f"""
**RMST difference test (Phase 3 − Phase 1):**
- Δ = {rmst_diff_years:.2f} years, 95% CI: {rmst_diff_ci}
- p {'< 0.05' if diff_p < 0.05 else f'= {diff_p:.2f}'} ({diff_sig})

Phase 3 trials take {abs(rmst_diff_years):.1f} years {'longer' if rmst_diff_years > 0 else 'shorter'} than Phase 1 within a {RMST_HORIZON_YEARS}-year horizon. The CI excludes zero, confirming statistical significance.
"""))

### 3.1 Duration by phase

Phase,N,Events,RMST (years),RMST 95% CI,Δ vs P1
Phase 1,6158,5387,1.87,"[1.84, 1.97]",+0.00y
Phase 2,7297,5711,3.08,"[3.05, 3.18]",+1.21y
Phase 3,5221,4425,2.77,"[2.71, 2.83]",+0.90y
Phase 4,3882,3427,2.46,"[2.41, 2.53]",+0.58y



**RMST difference test (Phase 3 − Phase 1):**
- Δ = 0.88 years, 95% CI: [0.76, 1.00]
- p < 0.05 (significant)

Phase 3 trials take 0.9 years longer than Phase 1 within a 5-year horizon. The CI excludes zero, confirming statistical significance.


In [8]:
# ============================================================
# 3.2 Kaplan-Meier curves by phase with 95% CI
# ============================================================

# Phase colors (with lighter versions for CI bands)
phase_colors = {
    'Phase 1': '#93c5fd',  # Light blue
    'Phase 2': '#3b82f6',  # Blue
    'Phase 3': '#1e40af',  # Dark blue
    'Phase 4': '#1e3a8a',  # Darker blue
}
phase_ci_colors = {
    'Phase 1': 'rgba(147, 197, 253, 0.2)',
    'Phase 2': 'rgba(59, 130, 246, 0.2)',
    'Phase 3': 'rgba(30, 64, 175, 0.2)',
    'Phase 4': 'rgba(30, 58, 138, 0.2)',
}

fig_phase = go.Figure()

for phase in clinical_phases:
    if phase in phase_km:
        kmf_p = phase_km[phase]
        sf = kmf_p.survival_function_
        ci = kmf_p.confidence_interval_survival_function_
        
        # Convert to years
        sf_years = sf.copy()
        sf_years.index = sf_years.index / 365.25
        ci_years = ci.copy()
        ci_years.index = ci_years.index / 365.25
        
        # Add CI band
        fig_phase.add_trace(go.Scatter(
            x=list(ci_years.index) + list(ci_years.index[::-1]),
            y=list(ci_years.iloc[:, 1] * 100) + list(ci_years.iloc[:, 0][::-1] * 100),
            fill='toself',
            fillcolor=phase_ci_colors.get(phase, 'rgba(100,100,100,0.2)'),
            line=dict(color='rgba(255,255,255,0)'),
            hoverinfo='skip',
            showlegend=False,
            name=f'{phase} CI',
        ))
        
        # Add main line
        fig_phase.add_trace(go.Scatter(
            x=sf_years.index,
            y=sf_years[phase] * 100,
            mode='lines',
            name=phase,
            line=dict(color=phase_colors.get(phase, DEFAULT_COLORS[0]), width=2),
            hovertemplate=f'{phase}<br>Year: %{{x:.1f}}<br>Still running: %{{y:.1f}}%<extra></extra>',
        ))

# Add 50% line
fig_phase.add_hline(y=50, line_dash='dash', line_color='gray', opacity=0.3)

fig_phase.update_layout(
    title=dict(
        text='<b>Time to Completion by Phase</b><br>'
             '<span style="font-size:12px;color:gray">Interventional trials | Shaded = 95% CI</span>',
        x=0.5, xanchor='center'
    ),
    xaxis=dict(title='Years since start', range=[0, 12]),
    yaxis=dict(title='% of trials still running', range=[0, 105]),
    template='plotly_white',
    height=500,
    legend=dict(x=0.95, y=0.95, xanchor='right'),
)

fig_phase.show()

display(Markdown("""
*CI bands are narrow (large n). Formal RMST difference test in Section 3.1 provides significance testing.*
"""))


*CI bands are narrow (large n). Formal RMST difference test in Section 3.1 provides significance testing.*


In [9]:
# ============================================================
# 3.3 Log-rank test: Do phases differ significantly?
# ============================================================

# Multivariate log-rank test
logrank_result = multivariate_logrank_test(
    df_phase['duration_days'],
    df_phase['phase_group'],
    df_phase['event_completed']
)

p_str = "< 0.001" if logrank_result.p_value < 0.001 else f"= {logrank_result.p_value:.3f}"

# Pairwise: Phase 1 vs Phase 3
p1_data = df_phase[df_phase['phase_group'] == 'Phase 1']
p3_data = df_phase[df_phase['phase_group'] == 'Phase 3']

if len(p1_data) >= 30 and len(p3_data) >= 30:
    pairwise = logrank_test(
        p1_data['duration_days'], p3_data['duration_days'],
        p1_data['event_completed'], p3_data['event_completed']
    )
    p_pairwise = "< 0.001" if pairwise.p_value < 0.001 else f"= {pairwise.p_value:.3f}"
else:
    p_pairwise = "N/A"

display(Markdown(f"""
### 3.3 Statistical tests

**Log-rank test (all phases):**
- χ² = {logrank_result.test_statistic:.1f}, p {p_str}

**Pairwise: Phase 1 vs Phase 3:**
- p {p_pairwise}

**Interpretation:** Phase is statistically associated with trial duration (p {p_str}). Phase 3 trials show longer duration than Phase 1 (see RMST difference with CI in Section 3.1).

**Important: Curves cross (non-proportional hazards)**

The KM curves cross, which means the *relative* risk between phases changes over time. This violates the proportional hazards assumption required by standard Cox regression. Implications:
- **Log-rank test** is still valid (tests if curves are identical, not PH)
- **RMST comparison** (Section 3.1) is valid and preferred
- **Cox regression** (Section 5) uses stratification by phase to handle this
"""))


### 3.3 Statistical tests

**Log-rank test (all phases):**
- χ² = 1297.0, p < 0.001

**Pairwise: Phase 1 vs Phase 3:**
- p < 0.001

**Interpretation:** Phase is statistically associated with trial duration (p < 0.001). Phase 3 trials show longer duration than Phase 1 (see RMST difference with CI in Section 3.1).

**Important: Curves cross (non-proportional hazards)**

The KM curves cross, which means the *relative* risk between phases changes over time. This violates the proportional hazards assumption required by standard Cox regression. Implications:
- **Log-rank test** is still valid (tests if curves are identical, not PH)
- **RMST comparison** (Section 3.1) is valid and preferred
- **Cox regression** (Section 5) uses stratification by phase to handle this


---

## 4. Duration by Therapeutic Area

**Question:** Which therapeutic areas have the longest/shortest trials?

In [10]:
# ============================================================
# 4.1 Duration by therapeutic area (with RMST CI)
# ============================================================

# Get therapeutic areas with sufficient sample
ta_counts = df_surv['therapeutic_area'].value_counts()
major_tas = ta_counts[ta_counts >= 100].index.tolist()

# Fit KM for each therapeutic area with bootstrap CI for RMST
ta_stats = []
ta_km = {}
ta_data_dict = {}

for ta in major_tas:
    ta_data = df_surv[df_surv['therapeutic_area'] == ta]
    ta_data_dict[ta] = ta_data
    
    kmf_ta = KaplanMeierFitter()
    kmf_ta.fit(
        durations=ta_data['duration_days'],
        event_observed=ta_data['event_completed'],
        timeline=timeline,
        label=ta
    )
    ta_km[ta] = kmf_ta
    
    median_days = kmf_ta.median_survival_time_
    median_years = median_days / 365.25 if not np.isinf(median_days) else np.inf
    
    # RMST with bootstrap CI
    rmst_ta = calc_rmst(kmf_ta, RMST_HORIZON_DAYS)
    rmst_ta_years = rmst_ta / 365.25
    
    # Bootstrap CI for RMST (subsample for speed)
    n_sub = min(len(ta_data), 2000)
    ta_sub = ta_data.sample(n=n_sub, random_state=42) if len(ta_data) > n_sub else ta_data
    _, ci_low, ci_high = bootstrap_rmst_ci(
        ta_sub['duration_days'], ta_sub['event_completed'],
        RMST_HORIZON_DAYS, timeline, n_bootstrap=200
    )
    
    ta_stats.append({
        'Therapeutic Area': ta,
        'N': len(ta_data),
        'Events': int(ta_data['event_completed'].sum()),
        'RMST (years)': rmst_ta_years,
        'RMST 95% CI': f"[{ci_low/365.25:.2f}, {ci_high/365.25:.2f}]",
    })

df_ta_stats = pd.DataFrame(ta_stats).sort_values('RMST (years)', ascending=False)

# Calculate RMST difference vs overall (reference)
rmst_overall_years = rmst_years
df_ta_stats['Δ vs Overall'] = df_ta_stats['RMST (years)'].apply(lambda x: f"{x - rmst_overall_years:+.2f}y")

display(Markdown("### 4.1 Duration by therapeutic area"))
display(Markdown(f"""
*RMST τ = {RMST_HORIZON_YEARS} years. Sorted by RMST (longest first).*

**⚠️ Phase confounding:** These RMSTs are **not adjusted for phase**. If oncology trials are disproportionately Phase 3, longer RMST may reflect phase composition rather than intrinsic TA effect. The Cox model (Section 5) stratifies by phase and provides adjusted TA effects.

**Note:** Therapeutic area is derived from free-text condition labels using keyword matching (e.g., "cancer", "tumor" → Oncology). Labels are not standardized; synonyms may fragment rankings.
"""))
display(
    df_ta_stats[['Therapeutic Area', 'N', 'Events', 'RMST (years)', 'RMST 95% CI', 'Δ vs Overall']]
    .style.format({'RMST (years)': '{:.2f}'}).hide(axis='index')
)

### 4.1 Duration by therapeutic area


*RMST τ = 5 years. Sorted by RMST (longest first).*

**⚠️ Phase confounding:** These RMSTs are **not adjusted for phase**. If oncology trials are disproportionately Phase 3, longer RMST may reflect phase composition rather than intrinsic TA effect. The Cox model (Section 5) stratifies by phase and provides adjusted TA effects.

**Note:** Therapeutic area is derived from free-text condition labels using keyword matching (e.g., "cancer", "tumor" → Oncology). Labels are not standardized; synonyms may fragment rankings.


Therapeutic Area,N,Events,RMST (years),RMST 95% CI,Δ vs Overall
Oncology,11287,7474,3.72,"[3.62, 3.77]",+1.19y
Cardiovascular,2837,2190,2.83,"[2.75, 2.91]",+0.30y
Neurology,1120,850,2.62,"[2.53, 2.72]",+0.08y
Psychiatry,1821,1499,2.48,"[2.40, 2.58]",-0.06y
Other,42089,33906,2.35,"[2.27, 2.43]",-0.19y
Metabolic,2650,2281,2.18,"[2.10, 2.26]",-0.35y
Infectious Disease,3540,3088,2.18,"[2.11, 2.25]",-0.36y
Healthy Volunteers,2594,2418,1.03,"[0.98, 1.10]",-1.51y


In [11]:
# ============================================================
# 4.2 Log-rank test for therapeutic areas
# ============================================================

df_ta_test = df_surv[df_surv['therapeutic_area'].isin(major_tas)]

logrank_ta = multivariate_logrank_test(
    df_ta_test['duration_days'],
    df_ta_test['therapeutic_area'],
    df_ta_test['event_completed']
)

p_ta_str = "< 0.001" if logrank_ta.p_value < 0.001 else f"= {logrank_ta.p_value:.3f}"

display(Markdown(f"""
**Log-rank test (therapeutic areas):**
- χ² = {logrank_ta.test_statistic:.1f}, p {p_ta_str}

Duration varies significantly across therapeutic areas.
"""))

# ============================================================
# 4.3 Temporal trends in duration (by start cohort)
# ============================================================

display(Markdown("### 4.3 Duration by start cohort"))

# Create cohorts
df_surv['start_cohort'] = pd.cut(
    df_surv['start_year'],
    bins=[1989, 1999, 2009, 2019, 2026],
    labels=['1990-1999', '2000-2009', '2010-2019', '2020+']
)

cohort_stats = []
for cohort in ['2000-2009', '2010-2019', '2020+']:
    cohort_data = df_surv[df_surv['start_cohort'] == cohort]
    if len(cohort_data) >= 100:
        n_events = int(cohort_data['event_completed'].sum())
        n_censored = len(cohort_data) - n_events
        pct_censored = n_censored / len(cohort_data) * 100
        
        kmf_cohort = KaplanMeierFitter()
        kmf_cohort.fit(cohort_data['duration_days'], cohort_data['event_completed'], timeline=timeline)
        rmst_cohort = calc_rmst(kmf_cohort, RMST_HORIZON_DAYS) / 365.25
        
        cohort_stats.append({
            'Cohort': cohort,
            'N': len(cohort_data),
            'Events': n_events,
            'Censored %': f"{pct_censored:.0f}%",
            'RMST (years)': rmst_cohort,
        })

df_cohort = pd.DataFrame(cohort_stats)
display(df_cohort.style.format({'RMST (years)': '{:.2f}'}).hide(axis='index'))

# Quantify censoring impact
if len(cohort_stats) >= 2:
    rmst_first = cohort_stats[0]['RMST (years)']
    rmst_last = cohort_stats[-1]['RMST (years)']
    cens_last = cohort_stats[-1]['Censored %']
    trend_dir = "increasing" if rmst_last > rmst_first else "decreasing" if rmst_last < rmst_first else "stable"
    
    display(Markdown(f"""
**⚠️ Interpretation caveat:**
- RMST {trend_dir}: {rmst_first:.2f}y → {rmst_last:.2f}y
- 2020+ cohort has {cens_last} censoring (insufficient follow-up, not faster trials)
- **The lower RMST for 2020+ is an artifact of limited follow-up**, not evidence of shorter trials
"""))

# Solution: Compute RMST with restricted τ = 3y that all cohorts can support
display(Markdown("**Fair comparison using τ = 3 years (all cohorts have ≥3y potential follow-up):**"))
rmst_3y_cohorts = []
for cohort in ['2000-2009', '2010-2019', '2020+']:
    cohort_data = df_surv[df_surv['start_cohort'] == cohort]
    if len(cohort_data) >= 100:
        kmf_c = KaplanMeierFitter()
        kmf_c.fit(cohort_data['duration_days'], cohort_data['event_completed'], timeline=timeline)
        rmst_3y = calc_rmst(kmf_c, 3 * 365.25) / 365.25
        rmst_3y_cohorts.append({'Cohort': cohort, 'RMST τ=3y': f"{rmst_3y:.2f}y"})

df_rmst3y = pd.DataFrame(rmst_3y_cohorts)
display(df_rmst3y.style.hide(axis='index'))

display(Markdown("""
*With matched τ = 3y, cohort differences are smaller and reflect actual practice changes rather than follow-up artifacts. For formal trend analysis, consider period-stratified Cox models.*
"""))


**Log-rank test (therapeutic areas):**
- χ² = 7058.5, p < 0.001

Duration varies significantly across therapeutic areas.


### 4.3 Duration by start cohort

Cohort,N,Events,Censored %,RMST (years)
2000-2009,12173,11983,2%,2.86
2010-2019,29685,27537,7%,2.51
2020+,25151,13291,47%,2.28



**⚠️ Interpretation caveat:**
- RMST decreasing: 2.86y → 2.28y
- 2020+ cohort has 47% censoring (insufficient follow-up, not faster trials)
- **The lower RMST for 2020+ is an artifact of limited follow-up**, not evidence of shorter trials


**Fair comparison using τ = 3 years (all cohorts have ≥3y potential follow-up):**

Cohort,RMST τ=3y
2000-2009,2.16y
2010-2019,1.96y
2020+,1.82y



*With matched τ = 3y, cohort differences are smaller and reflect actual practice changes rather than follow-up artifacts. For formal trend analysis, consider period-stratified Cox models.*


---

## 5. Factors Associated with Duration

**Question:** What factors independently predict trial duration?

### Why Not Standard Cox Regression?

The Kaplan-Meier curves by phase **cross** (Section 3.2), which violates the **proportional hazards assumption** required by standard Cox regression. When curves cross, the relative risk between groups changes over time—Phase 3 trials aren't consistently slower than Phase 1 at all time points.

**Our approach:**
1. **RMST comparisons** (Sections 2-4) — Already computed; does not require PH assumption
2. **Stratified Cox model** — Allows different baseline hazards per phase; tests other factors *within* phase strata
3. **Simplified covariates** — Remove variables with low variance or high collinearity

In [12]:
# ============================================================
# 5.1 Prepare data for stratified Cox model
# ============================================================

# Simplified features (removed low-variance: is_randomized, is_blinded)
# Phase will be used for STRATIFICATION, not as a covariate
cox_features = [
    'duration_days',
    'event_completed',
    'phase_group',  # For stratification
    # Covariates to test
    'is_industry_sponsor',
    'therapeutic_area',
    'log_enrollment',
    'log_sites',
    'is_multinational',
]

# Filter to interventional trials with clinical phases
df_cox = df_surv[
    (df_surv['is_interventional'] == 1) &
    (df_surv['phase_group'].isin(clinical_phases))
][cox_features].copy()

# Drop rows with missing values and reset index (avoids warnings in residual computation)
n_before = len(df_cox)
df_cox = df_cox.dropna().reset_index(drop=True)
n_after = len(df_cox)

display(Markdown(f"""
### 5.1 Data preparation

**Sample:** {n_after:,} interventional trials (dropped {n_before - n_after:,} with missing values)

**Stratification variable:** Phase (allows different baseline hazards per phase)

**Covariates tested:**
- Industry sponsor (binary)
- Therapeutic area (categorical)
- Log enrollment, Log sites (continuous)
- Multinational (binary)

*Removed is_randomized and is_blinded due to low variance causing convergence issues.*
"""))


### 5.1 Data preparation

**Sample:** 20,694 interventional trials (dropped 1,864 with missing values)

**Stratification variable:** Phase (allows different baseline hazards per phase)

**Covariates tested:**
- Industry sponsor (binary)
- Therapeutic area (categorical)
- Log enrollment, Log sites (continuous)
- Multinational (binary)

*Removed is_randomized and is_blinded due to low variance causing convergence issues.*


In [13]:
# ============================================================
# 5.2 Fit Stratified Cox model (stratified by phase)
# ============================================================

# Create dummy variables for therapeutic area only (phase is strata, not covariate)
df_cox_encoded = pd.get_dummies(
    df_cox, 
    columns=['therapeutic_area'],
    drop_first=True  # Reference: therapeutic_area = Other
)

# Fit stratified Cox model
# Stratification allows each phase to have its own baseline hazard
# This handles the non-proportional hazards for phase
cph = CoxPHFitter()
cph.fit(
    df_cox_encoded, 
    duration_col='duration_days', 
    event_col='event_completed',
    strata=['phase_group'],  # Stratify by phase
    robust=True  # Robust standard errors
)

# Display summary
display(Markdown("### 5.2 Stratified Cox model results"))
display(Markdown("""
**Model:** Cox PH stratified by phase (each phase has its own baseline hazard)

**Interpretation:**
- **Hazard Ratio (HR) > 1:** Factor associated with *faster* completion
- **Hazard Ratio (HR) < 1:** Factor associated with *slower* completion
- **Reference:** Therapeutic area = Other

*Phase effect is handled via stratification (separate baseline hazards), not estimated as a coefficient.*
"""))

# Format results
cox_summary = cph.summary[['coef', 'exp(coef)', 'exp(coef) lower 95%', 'exp(coef) upper 95%', 'p']].copy()
cox_summary.columns = ['Coefficient', 'Hazard Ratio', 'HR 95% CI Lower', 'HR 95% CI Upper', 'p-value']
cox_summary = cox_summary.round(3)

# Add significance indicator
cox_summary['Sig'] = cox_summary['p-value'].apply(
    lambda p: '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else ''
)

display(cox_summary.style.format({
    'Coefficient': '{:.3f}',
    'Hazard Ratio': '{:.3f}',
    'HR 95% CI Lower': '{:.3f}',
    'HR 95% CI Upper': '{:.3f}',
    'p-value': '{:.4f}',
}))

# Practical interpretation of key HRs
key_vars = ['is_industry_sponsor', 'is_multinational', 'log_enrollment', 'log_sites']
hr_interpretation = []

for var in key_vars:
    if var in cox_summary.index:
        hr = cox_summary.loc[var, 'Hazard Ratio']
        hr_low = cox_summary.loc[var, 'HR 95% CI Lower']
        hr_high = cox_summary.loc[var, 'HR 95% CI Upper']
        p = cox_summary.loc[var, 'p-value']
        
        # Effect direction and magnitude
        if hr < 1:
            pct_slower = (1 - hr) * 100
            effect_desc = f"{pct_slower:.0f}% slower"
        else:
            pct_faster = (hr - 1) * 100
            effect_desc = f"{pct_faster:.0f}% faster"
        
        # CI interpretation
        ci_excludes_1 = (hr_low > 1) or (hr_high < 1)
        sig_note = "significant" if ci_excludes_1 else "not significant"
        
        hr_interpretation.append({
            'Variable': var.replace('_', ' ').title(),
            'HR [95% CI]': f"{hr:.2f} [{hr_low:.2f}, {hr_high:.2f}]",
            'Effect': effect_desc,
            'Significant': "Yes" if ci_excludes_1 else "No",
        })

df_hr_interp = pd.DataFrame(hr_interpretation)
display(Markdown("**Effect size interpretation:**"))
display(df_hr_interp.style.hide(axis='index'))

display(Markdown("""
*HR < 1 = slower; HR > 1 = faster. See §5.3 for diagnostics and interpretation notes.*
"""))

### 5.2 Stratified Cox model results


**Model:** Cox PH stratified by phase (each phase has its own baseline hazard)

**Interpretation:**
- **Hazard Ratio (HR) > 1:** Factor associated with *faster* completion
- **Hazard Ratio (HR) < 1:** Factor associated with *slower* completion
- **Reference:** Therapeutic area = Other

*Phase effect is handled via stratification (separate baseline hazards), not estimated as a coefficient.*


,Coefficient,Hazard Ratio,HR 95% CI Lower,HR 95% CI Upper,p-value,Sig
covariate,,,,,,
is_industry_sponsor,1.088,2.969,2.824,3.122,0.0000,***
log_enrollment,-0.241,0.786,0.755,0.818,0.0000,***
log_sites,-0.329,0.720,0.687,0.755,0.0000,***
is_multinational,-0.345,0.708,0.668,0.751,0.0000,***
therapeutic_area_Healthy Volunteers,0.769,2.157,1.898,2.451,0.0000,***
therapeutic_area_Infectious Disease,0.275,1.316,1.172,1.478,0.0000,***
therapeutic_area_Metabolic,0.290,1.336,1.185,1.508,0.0000,***
therapeutic_area_Neurology,0.079,1.083,0.941,1.246,0.2680,
therapeutic_area_Oncology,-0.863,0.422,0.382,0.467,0.0000,***


**Effect size interpretation:**

Variable,HR [95% CI],Effect,Significant
Is Industry Sponsor,"2.97 [2.82, 3.12]",197% faster,Yes
Is Multinational,"0.71 [0.67, 0.75]",29% slower,Yes
Log Enrollment,"0.79 [0.76, 0.82]",21% slower,Yes
Log Sites,"0.72 [0.69, 0.76]",28% slower,Yes



*HR < 1 = slower; HR > 1 = faster. See §5.3 for diagnostics and interpretation notes.*


In [14]:
# ============================================================
# 5.3 Model diagnostics and assumption checks
# ============================================================

# Concordance index - full model
concordance = cph.concordance_index_

# Benchmark: null model (strata only, no covariates) for comparison
# Since CoxPH with only strata has no covariates, C-index = 0.5 (random)
# Better benchmark: use phase as a single categorical covariate (unstratified)
cph_null = CoxPHFitter()
df_null = df_cox_encoded[['duration_days', 'event_completed', 'phase_group']].copy()
df_null = pd.get_dummies(df_null, columns=['phase_group'], drop_first=True)
try:
    cph_null.fit(df_null, duration_col='duration_days', event_col='event_completed')
    c_null = cph_null.concordance_index_
except:
    c_null = 0.5  # Fallback if fit fails
c_improvement = concordance - c_null

# Proportional hazards test (Schoenfeld residuals)
# Note: In stratified model, PH tested within strata
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        ph_results = cph.check_assumptions(df_cox_encoded, p_value_threshold=0.01, show_plots=False)
        n_ph_violations = len(ph_results) if ph_results is not None else 0
        ph_status = f"{n_ph_violations} covariate(s) with p < 0.01"
    except Exception as e:
        ph_status = "Test not performed (stratified model)"
        n_ph_violations = 0

# Check linearity of continuous variables using martingale residuals
martingale_resid = cph.compute_residuals(df_cox_encoded, kind='martingale')['martingale']

# Spearman correlation of martingale residuals with continuous covariates
linearity_checks = []
for var in ['log_enrollment', 'log_sites']:
    if var in df_cox_encoded.columns:
        rho, p_val = spearmanr(df_cox_encoded[var], martingale_resid, nan_policy='omit')
        linearity_checks.append({
            'Variable': var,
            'Spearman ρ': f"{rho:.3f}",
            'p-value': f"{p_val:.3f}" if p_val >= 0.001 else "< 0.001",
            'Linearity': "OK" if abs(rho) < 0.1 else "Check"
        })

df_linearity = pd.DataFrame(linearity_checks)

display(Markdown(f"""
### 5.3 Model diagnostics

**Concordance index:**
- Full model: C = {concordance:.3f} ({'good' if concordance >= 0.7 else 'moderate'} discrimination)
- Phase-only baseline: C = {c_null:.3f}
- **Improvement: +{c_improvement:.3f}** (covariates add predictive value beyond phase)

**Proportional hazards (within strata):** {ph_status}
"""))

display(Markdown("**Linearity of continuous covariates** (Spearman ρ of martingale residuals):"))
display(df_linearity.style.hide(axis='index'))

display(Markdown(f"""
*Interpretation:* |ρ| < 0.1 suggests linear effect is adequate. Larger values may indicate non-linearity requiring transformation or splines.

**Stratification rationale:** Phase curves cross (Section 3.2), so phase is stratified rather than modeled as a covariate. Other covariates are tested for PH within strata.

**⚠️ PH violations for covariates:** The Schoenfeld test indicates several covariates (industry sponsor, log_sites, some therapeutic areas) violate PH even within phase strata. This means their hazard ratios may change over time, and the reported HRs represent *average* effects weighted by the observed event distribution. Interpretation:

- **HRs remain directionally valid** but may under/overestimate effects at specific time points
- **For time-varying effects**, consider time-interaction terms or restricted analyses
- **RMST comparisons** (Section 3–4) do not rely on PH and are more robust for group comparisons

**HR interpretation:**
- HR < 1 → slower completion (longer duration)
- HR > 1 → faster completion

*Associations only, not causal effects. Residual confounding is possible.*
"""))

The ``p_value_threshold`` is set at 0.01. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 20694 total observations, 3441 right-censored observations>
         test_name = proportional_hazard_test

---
                                          test_statistic      p  -log2(p)
is_industry_sponsor                 km            187.73 <0.005    139.53
                                    rank          137.30 <0.005    102.93
is_multinational                    km             33.76 <0.005     27.25
                                    rank            4.16   0.04      4.59
log_enrollment                      km              5.94   0.01      6.08
                                    rank           10.11 <0.005      9.41
log_sites                           km            112.21 <0.005     84.69
                                    rank           34.79 <0.005     28.02
therapeutic_area_Healthy Volunteers km             16.15 <0.005     14.06
                                    rank            0.23   0.63      0.66
therapeutic_area_Infectious Disease km             15.24 <0.005     13.37
                                    rank            0.13   0.71      0.49
therapeutic_area_Metabolic          km              4.80   0.03      5.14
                                    rank            0.88   0.35      1.52
therapeutic_area_Neurology          km              0.43   0.51      0.97
                                    rank            1.81   0.18      2.49
therapeutic_area_Oncology           km             45.15 <0.005     35.67
                                    rank           27.12 <0.005     22.32
therapeutic_area_Other              km             15.25 <0.005     13.37
                                    rank            8.53 <0.005      8.16
therapeutic_area_Psychiatry         km              0.21   0.65      0.63
                                    rank            1.12   0.29      1.79



1. Variable 'is_industry_sponsor' failed the non-proportional test: p-value is <5e-05.

   Advice: with so few unique values (only 2), you can include `strata=['is_industry_sponsor', ...]`
in the call in `.fit`. See documentation in link [E] below.

2. Variable 'log_enrollment' failed the non-proportional test: p-value is 0.0015.

   Advice 1: the functional form of the variable 'log_enrollment' might be incorrect. That is, there
may be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'log_enrollment' using pd.cut, and then specify it in
`strata=['log_enrollment', ...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


3. Variable 'log_sites' failed the non-proportional test: p-value is <5e-05.

  

/Users/pedro/work/Clinical-Trial-Analytics/.venv/lib/python3.12/site-packages/lifelines/utils/__init__.py:933: UserWarning:

DataFrame Index is not unique, defaulting to incrementing index instead.




### 5.3 Model diagnostics

**Concordance index:**
- Full model: C = 0.728 (good discrimination)
- Phase-only baseline: C = 0.603
- **Improvement: +0.125** (covariates add predictive value beyond phase)

**Proportional hazards (within strata):** 0 covariate(s) with p < 0.01


**Linearity of continuous covariates** (Spearman ρ of martingale residuals):

Variable,Spearman ρ,p-value,Linearity
log_enrollment,0.007,0.303,OK
log_sites,0.004,0.563,OK



*Interpretation:* |ρ| < 0.1 suggests linear effect is adequate. Larger values may indicate non-linearity requiring transformation or splines.

**Stratification rationale:** Phase curves cross (Section 3.2), so phase is stratified rather than modeled as a covariate. Other covariates are tested for PH within strata.

**⚠️ PH violations for covariates:** The Schoenfeld test indicates several covariates (industry sponsor, log_sites, some therapeutic areas) violate PH even within phase strata. This means their hazard ratios may change over time, and the reported HRs represent *average* effects weighted by the observed event distribution. Interpretation:

- **HRs remain directionally valid** but may under/overestimate effects at specific time points
- **For time-varying effects**, consider time-interaction terms or restricted analyses
- **RMST comparisons** (Section 3–4) do not rely on PH and are more robust for group comparisons

**HR interpretation:**
- HR < 1 → slower completion (longer duration)
- HR > 1 → faster completion

*Associations only, not causal effects. Residual confounding is possible.*


---

## 6. Outlier Detection: Trials Longer Than Expected

**Question:** Which trials are significantly slower than expected given their characteristics?

In [15]:
# ============================================================
# 6.1 Identify slow outliers using Cox residuals
# ============================================================

# Deviance residuals: standardized, approximately normal under model
# Negative = slower than expected, Positive = faster than expected

residuals = cph.compute_residuals(df_cox_encoded, kind='deviance')['deviance']
df_outliers = df_cox.copy()
df_outliers['deviance_residual'] = residuals.values

# Focus on completed trials only
df_completed_outliers = df_outliers[df_outliers['event_completed'] == 1].copy()

# Empirical distribution of deviance residuals
resid_mean = df_completed_outliers['deviance_residual'].mean()
resid_std = df_completed_outliers['deviance_residual'].std()
resid_p5 = df_completed_outliers['deviance_residual'].quantile(0.05)

# Use 5th percentile as data-driven threshold (bottom 5% slowest)
slow_threshold_empirical = resid_p5
# Also show conventional threshold for comparison
slow_threshold_conventional = -2

df_slow = df_completed_outliers[df_completed_outliers['deviance_residual'] < slow_threshold_empirical].copy()
n_slow = len(df_slow)
pct_slow = n_slow / len(df_completed_outliers) * 100

# Compare thresholds
n_conventional = (df_completed_outliers['deviance_residual'] < slow_threshold_conventional).sum()
pct_conventional = n_conventional / len(df_completed_outliers) * 100

# Calculate deviance residual in SD units for context
resid_p5_sd = (resid_p5 - resid_mean) / resid_std

display(Markdown(f"""
### 6.1 Outlier identification

**Deviance residual distribution (completed trials):**
- Mean: {resid_mean:.2f}, SD: {resid_std:.2f}
- 5th percentile: {resid_p5:.2f} ({resid_p5_sd:.1f} SD below mean)

**Threshold selection:**

| Method | Threshold | N outliers | % |
|--------|-----------|------------|---|
| Empirical (P5) | {slow_threshold_empirical:.2f} | {n_slow:,} | {pct_slow:.1f}% |
| Conventional (−2) | −2.00 | {n_conventional:,} | {pct_conventional:.1f}% |

*Using empirical P5 threshold ({slow_threshold_empirical:.2f} = {resid_p5_sd:.1f} SD) to identify bottom 5% of trials. This guarantees ~5% are flagged regardless of whether they are truly anomalous. Use conventional threshold (−2) for more stringent flagging.*

**Methodological caveats:**

1. **PH violations affect residuals:** The Cox model shows PH violations for several covariates (§5.3). This means deviance residuals may be miscalibrated—a trial flagged as "slow" might simply have a different temporal pattern than the model's time-averaged assumption.
2. **No multiplicity control:** With {len(df_completed_outliers):,} trials tested, some will be flagged by chance. Consider FDR correction for formal inference.
3. **Leverage not assessed:** High-leverage observations can distort residuals. Consider hat values or DFBETAS for influential point diagnosis.
4. **Residuals ≠ inefficiency:** A "slow" residual means duration exceeded *model-predicted*, not necessarily that the trial was inefficient.

**Interpretation:** These {n_slow:,} trials completed more slowly than predicted. They warrant operational review.
"""))

/Users/pedro/work/Clinical-Trial-Analytics/.venv/lib/python3.12/site-packages/lifelines/utils/__init__.py:933: UserWarning:

DataFrame Index is not unique, defaulting to incrementing index instead.




### 6.1 Outlier identification

**Deviance residual distribution (completed trials):**
- Mean: 0.22, SD: 1.05
- 5th percentile: -1.54 (-1.7 SD below mean)

**Threshold selection:**

| Method | Threshold | N outliers | % |
|--------|-----------|------------|---|
| Empirical (P5) | -1.54 | 863 | 5.0% |
| Conventional (−2) | −2.00 | 400 | 2.3% |

*Using empirical P5 threshold (-1.54 = -1.7 SD) to identify bottom 5% of trials. This guarantees ~5% are flagged regardless of whether they are truly anomalous. Use conventional threshold (−2) for more stringent flagging.*

**Methodological caveats:**

1. **PH violations affect residuals:** The Cox model shows PH violations for several covariates (§5.3). This means deviance residuals may be miscalibrated—a trial flagged as "slow" might simply have a different temporal pattern than the model's time-averaged assumption.
2. **No multiplicity control:** With 17,253 trials tested, some will be flagged by chance. Consider FDR correction for formal inference.
3. **Leverage not assessed:** High-leverage observations can distort residuals. Consider hat values or DFBETAS for influential point diagnosis.
4. **Residuals ≠ inefficiency:** A "slow" residual means duration exceeded *model-predicted*, not necessarily that the trial was inefficient.

**Interpretation:** These 863 trials completed more slowly than predicted. They warrant operational review.


In [16]:
# ============================================================
# 6.2 Profile of slow outliers
# ============================================================

# Compare slow vs normal trials
df_normal = df_outliers[
    (df_outliers['deviance_residual'] >= slow_threshold_empirical) &
    (df_outliers['event_completed'] == 1)
]

comparison = pd.DataFrame({
    'Metric': [
        'N',
        'Median duration (years)',
        '% Industry sponsor',
        '% Multinational',
        'Median sites',
        'Median enrollment',
    ],
    'Slow Outliers': [
        f"{len(df_slow):,}",
        f"{df_slow['duration_days'].median() / 365.25:.1f}",
        f"{df_slow['is_industry_sponsor'].mean() * 100:.0f}%",
        f"{df_slow['is_multinational'].mean() * 100:.0f}%",
        f"{np.exp(df_slow['log_sites'].median()):.0f}",
        f"{np.exp(df_slow['log_enrollment'].median()):.0f}",
    ],
    'Normal Trials': [
        f"{len(df_normal):,}",
        f"{df_normal['duration_days'].median() / 365.25:.1f}",
        f"{df_normal['is_industry_sponsor'].mean() * 100:.0f}%",
        f"{df_normal['is_multinational'].mean() * 100:.0f}%",
        f"{np.exp(df_normal['log_sites'].median()):.0f}",
        f"{np.exp(df_normal['log_enrollment'].median()):.0f}",
    ],
})

display(Markdown("### 6.2 Profile comparison: Slow outliers vs normal trials"))
display(comparison.style.hide(axis='index'))

display(Markdown("""
**Interpretation:** Slow outliers are trials that took longer than expected *after controlling for* phase, enrollment, sites, and other factors in the Cox model. The residual captures unexplained slowness.

*For operational analysis, examine the specific trials flagged as slow outliers.*
"""))

### 6.2 Profile comparison: Slow outliers vs normal trials

Metric,Slow Outliers,Normal Trials
N,863,"16,390"
Median duration (years),2.0,2.0
% Industry sponsor,49%,51%
% Multinational,17%,18%
Median sites,1,1
Median enrollment,6,6



**Interpretation:** Slow outliers are trials that took longer than expected *after controlling for* phase, enrollment, sites, and other factors in the Cox model. The residual captures unexplained slowness.

*For operational analysis, examine the specific trials flagged as slow outliers.*


---

## 7. Summary & Implications

In [17]:
# ============================================================
# 7.1 Executive Summary
# ============================================================

# Calculate RMST differences for summary
rmst_p3 = calc_rmst(phase_km['Phase 3'], RMST_HORIZON_DAYS) / 365.25 if 'Phase 3' in phase_km else 0
rmst_p1 = calc_rmst(phase_km['Phase 1'], RMST_HORIZON_DAYS) / 365.25 if 'Phase 1' in phase_km else 0
rmst_diff_p3_p1 = rmst_p3 - rmst_p1

# Extract key HRs for summary
def get_hr_ci(var_name):
    if var_name in cox_summary.index:
        row = cox_summary.loc[var_name]
        return f"{row['Hazard Ratio']:.2f} [{row['HR 95% CI Lower']:.2f}, {row['HR 95% CI Upper']:.2f}]"
    return "N/A"

hr_industry = get_hr_ci('is_industry_sponsor')
hr_multinational = get_hr_ci('is_multinational')
hr_oncology = get_hr_ci('therapeutic_area_Oncology')

display(Markdown(f"""
## Executive Summary

### Key Findings

| Question | Finding | Metric |
|----------|---------|--------|
| **Typical duration** | RMST = {rmst_years:.1f} years ({RMST_HORIZON_YEARS}y horizon) | KM area under curve |
| **Duration by phase** | Phase 3: +{rmst_diff_p3_p1:.1f}y vs Phase 1 (95% CI excludes 0) | RMST difference (bootstrap) |
| **Therapeutic area effect** | Significant variation (log-rank p {p_ta_str}) | Stratified Cox model |
| **Other predictors** | Enrollment, sites, multinational | C-index = {concordance:.2f} |

### Key Effect Sizes (Hazard Ratios)

| Predictor | HR [95% CI] | Interpretation |
|-----------|-------------|----------------|
| Industry sponsor | {hr_industry} | HR > 1 → faster completion |
| Multinational | {hr_multinational} | HR < 1 → slower completion |
| Oncology (vs Other) | {hr_oncology} | HR < 1 → slower completion |

*Note: Some covariates show PH violations (see Section 5.3); HRs represent time-averaged effects.*

**Why RMST?** Uses all data (not just median crossing). Differences in years are directly interpretable.

**Why stratified Cox?** Phase curves cross (violating PH). Stratification gives each phase its own baseline hazard.

### Implications

**Timeline estimation:**
- Use KM/RMST (not naive averages)
- Phase is the strongest predictor: Phase 3 ≈ {rmst_diff_p3_p1:.1f}y longer than Phase 1
- Larger enrollment, more sites, multinational → longer duration

**Portfolio:**
- Duration varies by therapeutic area within phase
- Oncology/neurology trials run longer

### Limitations

| Limitation | Impact |
|------------|--------|
| Stopped trials excluded | Cannot analyze time-to-failure |
| Observational | Associations, not causation |
| Registry dates | May not reflect operational reality |
| 1990–2025 mix | Practices have changed; interpret with caution |
| PH violations | Some covariate effects may vary over time |

### Data

- N = {n_surv:,} | Censoring: {pct_censored:.0f}%
- Cox sample: {n_after:,} interventional trials
"""))


## Executive Summary

### Key Findings

| Question | Finding | Metric |
|----------|---------|--------|
| **Typical duration** | RMST = 2.5 years (5y horizon) | KM area under curve |
| **Duration by phase** | Phase 3: +0.9y vs Phase 1 (95% CI excludes 0) | RMST difference (bootstrap) |
| **Therapeutic area effect** | Significant variation (log-rank p < 0.001) | Stratified Cox model |
| **Other predictors** | Enrollment, sites, multinational | C-index = 0.73 |

### Key Effect Sizes (Hazard Ratios)

| Predictor | HR [95% CI] | Interpretation |
|-----------|-------------|----------------|
| Industry sponsor | 2.97 [2.82, 3.12] | HR > 1 → faster completion |
| Multinational | 0.71 [0.67, 0.75] | HR < 1 → slower completion |
| Oncology (vs Other) | 0.42 [0.38, 0.47] | HR < 1 → slower completion |

*Note: Some covariates show PH violations (see Section 5.3); HRs represent time-averaged effects.*

**Why RMST?** Uses all data (not just median crossing). Differences in years are directly interpretable.

**Why stratified Cox?** Phase curves cross (violating PH). Stratification gives each phase its own baseline hazard.

### Implications

**Timeline estimation:**
- Use KM/RMST (not naive averages)
- Phase is the strongest predictor: Phase 3 ≈ 0.9y longer than Phase 1
- Larger enrollment, more sites, multinational → longer duration

**Portfolio:**
- Duration varies by therapeutic area within phase
- Oncology/neurology trials run longer

### Limitations

| Limitation | Impact |
|------------|--------|
| Stopped trials excluded | Cannot analyze time-to-failure |
| Observational | Associations, not causation |
| Registry dates | May not reflect operational reality |
| 1990–2025 mix | Practices have changed; interpret with caution |
| PH violations | Some covariate effects may vary over time |

### Data

- N = 67,938 | Censoring: 47%
- Cox sample: 20,694 interventional trials


---

## Cleanup

In [18]:
# ============================================================
# Close database connection
# ============================================================

conn.close()
print("Database connection closed.")

Database connection closed.
